# Datawrapper API: Team analytics

#### Load Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
from datawrapper import Datawrapper
import altair as alt
import altair_stiles as altstiles

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

#### Auth

In [5]:
token = !echo $dw_api
dw = Datawrapper(access_token=token[0])

#### Account information

In [6]:
dw.account_info()

{'id': 393404,
 'email': 'mstiles@mipo.news',
 'name': None,
 'role': 'editor',
 'language': 'en-US',
 'teams': [{'id': '1NhUBkSc',
   'name': 'Grid',
   'url': '/v3/teams/1NhUBkSc',
   'active': True}],
 'chartCount': 231,
 'url': '/v3/users/393404',
 'activeTeam': '1NhUBkSc'}

#### Get a list of our charts

In [7]:
charts = dw.get_charts(limit=1000)

#### How many?

In [8]:
len(charts)

455

#### Our team's folders

In [9]:
dw.get_folders()

{'status': 'ok',
 'data': [{'type': 'user', 'id': 393404, 'charts': 4, 'folders': []},
  {'type': 'team',
   'id': '1NhUBkSc',
   'name': 'Grid',
   'charts': 280,
   'folders': [{'id': 86018,
     'name': 'ready-to-run',
     'folders': [],
     'charts': 337},
    {'id': 86794, 'name': 'needs-edit', 'folders': [], 'charts': 1},
    {'id': 87068, 'name': 'not-using?', 'folders': [], 'charts': 308},
    {'id': 105399, 'name': 'war-in-data', 'folders': [], 'charts': 7}]}]}

#### Read the list as a dataframe

In [10]:
src = pd.DataFrame(charts)

In [11]:
len(src)

455

In [12]:
src["folderId"] = (
    src["folderId"].fillna("").astype(str).str.replace(".0", "", regex=False)
)

In [13]:
src["authorId"] = src["authorId"].astype(str)

#### Filter dataframe to just the 'ready to run' charts

In [14]:
df = src[src["folderId"] == "86018"].copy()

#### All charts

In [15]:
# df = src.copy()

---

#### How many?

In [16]:
len(df)

327

In [17]:
df.folderId.value_counts()

86018    327
Name: folderId, dtype: int64

#### Fill in team members' names for codes

In [18]:
contributors = {
    "393403": "Matthews",
    "393404": "Stiles",
    "409023": "Deen",
    "243254": "Kwon",
    "396758": "Levitan",
}

In [19]:
df["author_name"] = df["authorId"].map(contributors)

#### Frequencies by reporter on 'ready to run' charts

In [20]:
df.author_name.value_counts()

Stiles      115
Deen        111
Matthews     89
Kwon          7
Levitan       5
Name: author_name, dtype: int64

In [21]:
authors = df.author_name.value_counts().reset_index()

In [22]:
authors.rename(
    columns={"index": "author", "author_name": "ready_to_run_count"}, inplace=True
)

In [23]:
authors["share"] = (
    (authors["ready_to_run_count"] / authors["ready_to_run_count"].sum()) * 100
).round()

In [24]:
authors

,author,ready_to_run_count,share
0,Stiles,115,35.0
1,Deen,111,34.0
2,Matthews,89,27.0
3,Kwon,7,2.0
4,Levitan,5,2.0


#### Types

In [25]:
types = (
    df.groupby(["type"])
    .agg({"id": "size"})
    .reset_index()
    .sort_values("id", ascending=False)
)

In [26]:
types.rename(columns={"type": "chart_type", "id": "ready_to_run_count"}, inplace=True)

In [27]:
types

,chart_type,ready_to_run_count
9,d3-lines,81
7,d3-bars-stacked,44
10,d3-maps-choropleth,41
3,d3-bars,38
1,d3-area,22
0,column-chart,22
16,locator-map,17
18,tables,11
4,d3-bars-bullet,10
5,d3-bars-grouped,8


---

#### Get image url

In [28]:
df[["full", "plain"]] = pd.json_normalize(df["thumbnails"])

In [29]:
df.drop(["plain"], axis=1, inplace=True)

In [30]:
df.rename(columns={"full": "image_url"}, inplace=True)

---

#### Dates

In [31]:
df["datetime"] = pd.to_datetime(df["createdAt"]).dt.normalize()
df["date"] = df["datetime"].dt.date
df["month"] = df["datetime"].dt.month_name()

In [32]:
df["week_start"] = df["datetime"] - pd.to_timedelta(
    df["datetime"].dt.dayofweek, unit="d"
)

In [33]:
df["week_start"] = df["week_start"].dt.date

In [34]:
days = df.groupby(["date"]).agg({"id": "count"}).reset_index()
months = df.groupby(["month"]).agg({"id": "count"}).reset_index()
weeks = df.groupby(["week_start"]).agg({"id": "count"}).reset_index()

In [35]:
weeks_team = (
    df.groupby(["week_start", "author_name"]).agg({"id": "count"}).reset_index()
)

In [36]:
days.rename(columns={"id": "count"}, inplace=True)
months.rename(columns={"id": "count"}, inplace=True)
weeks.rename(columns={"id": "count", "week_start": "week"}, inplace=True)
weeks_team.rename(columns={"id": "count", "week_start": "week"}, inplace=True)

In [37]:
days["date"] = pd.to_datetime(days["date"])
weeks["week"] = pd.to_datetime(weeks["week"])
weeks_team["week"] = pd.to_datetime(weeks_team["week"])

In [38]:
core_team = ["Deen", "Matthews", "Stiles"]

In [39]:
alt.Chart(weeks_team[weeks_team["author_name"].isin(core_team)]).mark_bar(
    size=10
).encode(
    x=alt.X("week:T", title=""),
    y=alt.Y("count:Q", title=""),
    facet=alt.Facet("author_name", columns=1, title=" "),
).properties(
    width=600, height=100, title="Datawrapper charts, by week"
)

alt.Chart(...)

In [40]:
alt.Chart(weeks).mark_bar(size=10).encode(
    x=alt.X("week:T", title=""),
    y=alt.Y("count:Q", title=""),
).properties(width=600, height=300, title="Datawrapper charts, by week")

alt.Chart(...)

---

#### Charts created, by week

In [41]:
weeks["count"].mean()

8.605263157894736

In [42]:
weeks.head()

,week,count
0,2021-12-06,3
1,2021-12-13,11
2,2021-12-20,7
3,2021-12-27,6
4,2022-01-03,12


#### Export chart images

In [43]:
# for code, title, date in zip(df["id"], df["title"], df["date"]):
#     chart_id = f"{code}"
#     output = "png"
#     width = 360
#     filepath = f"images/{date}-{title.lower().replace(' ', '-')}-{code}.png"
#     dw.export_chart(chart_id=chart_id, output=output, width=width, filepath=filepath)

#### Just the important columns

In [44]:
df_slim = df[
    [
        "id",
        "title",
        "type",
        "author_name",
        "image_url",
        "date",
        "month",
        "week_start",
    ]
]

In [45]:
df_slim.head()

,id,title,type,author_name,image_url,date,month,week_start
1,ZCspC,California leads on electric vehicles,d3-maps-choropleth,Stiles,//img.datawrapper.de/ksTqI/ad6c0c28fb48c763a54843146f33d18b/full.png,2022-08-24,August,2022-08-22
3,ksTqI,California still fueled by gas,d3-bars,Stiles,//img.datawrapper.de/Q91BW/5ea61e53e0b6b3c100f337a7591f387e/full.png,2022-08-24,August,2022-08-22
4,iSNsf,School book bans by state,d3-maps-choropleth,Deen,//img.datawrapper.de/PdsBn/46639260a01abb52d2db1e82629d27b5/full.png,2022-08-23,August,2022-08-22
5,Q91BW,Dining out vs. eating at home,d3-lines,Deen,//img.datawrapper.de/zXfEf/4daf2cfeee1967898e5c084b2e1cdec1/full.png,2022-08-23,August,2022-08-22
6,PdsBn,Percent change in CPI for food,d3-lines,Deen,//img.datawrapper.de/A5jS1/5535716089c901a50952c0ab3b0c6e45/full.png,2022-08-23,August,2022-08-22


---

#### Export

In [46]:
df_slim.to_csv("data/processed/datawrapper_charts_all.csv", index=False)

In [47]:
months.to_csv("data/processed/datawrapper_charts_created_by_month.csv", index=False)

In [48]:
weeks.to_csv("data/processed/datawrapper_charts_created_by_week.csv", index=False)

In [49]:
days.to_csv("data/processed/datawrapper_charts_created_by_day.csv", index=False)

In [50]:
authors.to_csv("data/processed/datawrapper_charts_created_by_author.csv", index=False)

In [51]:
types.to_csv("data/processed/datawrapper_charts_created_by_type.csv", index=False)

In [52]:
df_slim.sample(6)

,id,title,type,author_name,image_url,date,month,week_start
386,wHpZs,Support for legal abortion by trimester,d3-bars-stacked,Matthews,NaN,2022-01-17,January,2022-01-17
372,8DPiU,Travel distance to nearest abortion provider,d3-maps-choropleth,Matthews,NaN,2022-01-20,January,2022-01-17
148,J4kV9,Children killed in shootings each year,column-chart,Deen,//img.datawrapper.de/wKP7v/4d1d00f5aac08c04f9452658800aae82/full.png,2022-05-24,May,2022-05-23
375,2tX5W,Change in home price index,d3-lines,Matthews,NaN,2022-01-18,January,2022-01-17
248,8o5Z6,2020 greenhouse gas disclosures from Chinese companies,d3-bars,Deen,//img.datawrapper.de/6a1YX/c8a9d8de287fc0ec9fc70d0e177d3372/full.png,2022-03-28,March,2022-03-28
355,f3VRV,Abortion access under Roe,d3-maps-choropleth,Matthews,NaN,2022-01-28,January,2022-01-24
